In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
sys.path.append('..')
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
from scipy.stats import beta
import warnings
from src.lesion import perform_lesion_experiment, do_lesion_hypo_tests
from src.pointers import DATA_PATHS
from src.experiment_tagging import get_model_path
from src.utils import bates_quantile
from scipy.stats import sem

Using TensorFlow backend.


In [2]:
models = [('mnist', 'CNN-MNIST'),
          # ('fashion', 'CNN-FASHION'),
          # ('cifar10', 'CNN-CIFAR10'),
          # ('mnist', 'CNN-MNIST+DROPOUT'), ('cifar10', 'CNN-CIFAR10+DROPOUT'),
          # ('fashion', 'CNN-FASHION+DROPOUT')
          ]

n_clust = 16
n_shuffles = 19
n_workers = 5
n_reps = 5
is_unpruned = True
results_dir = '/project/nn_clustering/results/'

In [3]:
all_results = []

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for dataset_name, tag in tqdm(models):
        for use_activations in [False, True]:
            for do_local in [False, True]:
                paths = get_model_path(tag, filter_='all')[-n_reps:]

                net_pkl_results = []
                fisher_p_means, chi2_p_means, effect_means = [], [], []
                fisher_p_ranges, chi2_p_ranges, effect_ranges = [], [], []
                fisher_stat_means, fisher_stat_ranges = [], []

                for path in paths:

                    (true_results,
                     all_random_results,
                     metadata,
                     evaluation) = perform_lesion_experiment('.' + DATA_PATHS[dataset_name],
                                                             path, n_clusters=n_clust,
                                                             n_shuffles=n_shuffles,
                                                             unpruned=is_unpruned,
                                                             activations=use_activations,
                                                             local=do_local)
                    net_pkl_results.append({'true_results': true_results,
                                            'all_random_results': all_random_results,
                                            'metadata': metadata,
                                            'evaluation': evaluation})
                    hypo_results = do_lesion_hypo_tests(evaluation, true_results, all_random_results)

                    fisher_p_means.append(hypo_results['fisher_p_means'])
                    fisher_stat_means.append(hypo_results['fisher_stat_means'])
                    chi2_p_means.append(hypo_results['chi2_p_means'])
                    effect_means.append(hypo_results['effect_factors_means'])
                    fisher_p_ranges.append(hypo_results['fisher_p_ranges'])
                    fisher_stat_ranges.append(hypo_results['fisher_stat_ranges'])
                    chi2_p_ranges.append(hypo_results['chi2_p_ranges'])
                    effect_ranges.append(hypo_results['effect_factors_range'])

                with open(results_dir + '/lesion_data_' + tag +
                          f'_activations={use_activations}_local={do_local}.pkl', 'wb') as f:
                    pickle.dump(net_pkl_results, f)

                model_results = {'is_unpruned': is_unpruned,
                                 'model_tag': tag,
                                 'activations': use_activations,
                                 'local': do_local,
                                 'fisher_p_means': bates_quantile(np.mean(np.array(fisher_p_means)), n_reps),
                                 'fisher_stat_means': np.mean(np.array(fisher_stat_means)),
                                 # 'chi2_p_means': bates_quantile(np.mean(np.array(chi2_p_means)), n_reps),
                                 'effect_means': np.mean(np.concatenate(effect_means))*2,
                                 'effect_means_sem': sem(np.concatenate(effect_means)*2, axis=None),
                                 'fisher_p_ranges': bates_quantile(np.mean(np.array(fisher_p_ranges)), n_reps),
                                 'fisher_stat_range': np.mean(np.array(fisher_stat_ranges)),
                                 # 'chi2_p_ranges': bates_quantile(np.mean(np.array(chi2_p_ranges)), n_reps),
                                 'effect_ranges': np.mean(np.concatenate(effect_ranges))*2,
                                 'effect_ranges_sem': sem(np.concatenate(effect_ranges)*2, axis=None),}
                all_results.append(pd.Series(model_results))

# result_df = pd.DataFrame(all_results)
# savepath = '../results/lesion_results_cnn.csv'
# result_df.to_csv(savepath)
# result_df


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [50:37<50:37, 3037.83s/it]

100%|██████████| 2/2 [1:46:13<00:00, 3127.14s/it]

100%|██████████| 2/2 [1:46:13<00:00, 3186.68s/it]

,is_unpruned,model_tag,dataset,chi2_p_means,chi2_p_ranges,combined_p_means,effect_ranges,effect_means,combined_p_ranges
0,True,CNN-MNIST,mnist,0.002164,0.892888,0.017104,5.508326e+00,1.000243,0.707797
1,True,CNN-MNIST,mnist,0.264915,0.322700,0.036717,4.033645e+00,1.000161,0.479884
2,True,CNN-MNIST,mnist,0.036334,0.297542,0.013149,2.412596e+00,1.000218,0.094653
3,True,CNN-MNIST,mnist,0.026125,0.561076,0.000428,6.390199e+00,1.000393,0.986759
4,True,CNN-MNIST,mnist,0.035189,0.877384,0.009793,5.987729e+00,1.000299,0.305394
5,True,CNN-FASHION,fashion,0.483188,0.787735,0.041504,inf,1.001231,0.737606
6,True,CNN-FASHION,fashion,0.000030,0.282699,0.000167,inf,1.001120,0.808584
7,True,CNN-FASHION,fashion,0.075078,0.223803,0.025246,1.676627e+11,1.001132,0.934899
8,True,CNN-FASHION,fashion,0.595642,0.282699,0.011908,4.411689e+11,1.001306,0.580515
9,True,CNN-FASHION,fashion,0.752382,0.879894,0.041674,inf,1.001280,0.304067
